In [2]:
import pandas as pd
import numpy as np
from src.utils.elo import Elo

## WIP - I want to calculate ELO ratings and add these as a a features into gameweek data

In [3]:
gameweek_df = pd.read_csv('data/processed/combined_gameweeks.csv')

/opt/homebrew/Caskroom/miniconda/base/envs/ravelin36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (25,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
teams = pd.read_csv(f'data/Fantasy-Premier-League/data/master_team_list.csv').rename(columns={'team':'opponent_team','team_name':'opponent_name'})

with_opponents = gameweek_df.merge(teams,on=['season','opponent_team'],how='left')

fixtures = with_opponents[['team','opponent_name','season','gw','team_h_score','team_a_score', 'was_home']].loc[with_opponents.team.notna()].drop_duplicates().copy()

fixtures['opp_team'] = 'opp_' + fixtures.opponent_name # so we can merge and update

fixtures['home_outcome'] =(np.clip(fixtures['team_h_score'] -fixtures['team_a_score'],-1,1) + 1) / 2

fixtures['outcome'] = np.where(fixtures.was_home, fixtures.home_outcome, 1-fixtures.home_outcome) # there must be a simpler way

elo =Elo(fixtures,k_factor=20, team_h='team',team_a='opp_team')
historic_elo, elo_ratings =elo.process_all_fixtures()

historic_elo[['team', 'season', 'gw','elo_e']]

In [20]:
pd.Series(elo_ratings).sort_values(ascending=False)

Man City              1351.724294
opp_Man City          1351.594701
opp_Chelsea           1316.792520
Chelsea               1316.360430
Liverpool             1309.572778
opp_Liverpool         1309.099231
opp_West Ham          1274.154114
West Ham              1274.026227
opp_Man Utd           1267.280718
Man Utd               1266.626007
opp_Arsenal           1258.882917
Arsenal               1258.857611
Spurs                 1233.906867
opp_Spurs             1233.660111
opp_Leicester         1224.651170
Leicester             1224.498526
Leeds                 1200.259701
opp_Leeds             1200.195909
opp_Everton           1198.083824
Everton               1197.901629
Wolves                1188.067737
opp_Wolves            1188.007969
Crystal Palace        1187.337366
opp_Crystal Palace    1187.254727
Brighton              1186.001063
opp_Brighton          1185.973330
opp_Aston Villa       1182.043647
Aston Villa           1181.254954
Brentford             1178.518200
opp_Brentford 